In [14]:
import requests
import time
import asyncio
import aiohttp
import multiprocessing

In [ ]:
#https://stackoverflow.com/questions/57126286/fastest-parallel-requests-in-python
async def get(url, session):
    try:
        async with session.get(url=url) as response:
            resp = await response.read()
            #print("Successfully got url {} with resp of length {}.".format(url, len(resp)))
    except Exception as e:
        print("Unable to get url {} due to {}.".format(url, e.__class__))


async def main(urls):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*(get(url, session) for url in urls))
    
    #print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))

test_websites = ['https://google.com'] * 100
start = time.time()
await main(test_websites)
end = time.time()
print(end - start)

start = time.time()
for w in test_websites:
    requests.get(w)
end = time.time()
print(end - start)

0.9386866092681885
37.71987056732178


In [32]:
import concurrent
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [ ]:
#https://stackoverflow.com/questions/57126286/fastest-parallel-requests-in-python
test_websites = ['https://google.com'] * 500
start = time.time()
await main(test_websites)
end = time.time()
print(end - start)


n_threads = 16

start = time.time()
with ProcessPoolExecutor(max_workers=n_threads) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(requests.get, web): web for web in test_websites}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            pass
            #print('%r page is %d bytes' % (url, len(data.content)))
end = time.time()
print(end - start)

# https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.ThreadPoolExecutor
start = time.time()

with ThreadPoolExecutor(max_workers=n_threads) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(requests.get, web): web for web in test_websites}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            pass
            #print('%r page is %d bytes' % (url, len(data.content)))
end = time.time()
print(end - start)

1.6224572658538818
23.54202365875244
21.242860794067383
